## Setup

To complete the following guide you will need to install the following packages:
- fireworks-ai
- pandas

You will also need:

- Fireworks account
- Fireworks API key

In [1]:
!pip install -q fireworks-ai pandas

In [29]:
import json
import os
import random
from collections import Counter
from sklearn.metrics import classification_report

from fireworks.client import Fireworks
import pandas as pd

In [3]:
# Make sure you have the FIREWORKS_API_KEY environment variable set to your account's key!
credentials = json.load(open("../credentials.json", "r"))
os.environ['FIREWORKS_API_KEY'] = credentials['fireworks_api_key']

client = Fireworks(api_key = credentials['fireworks_api_key'])

## Problem Definition: Insurance Support Ticket Classifier

*Note: The problem definition, data, and labels used in this example were synthetically generated by Claude 3 Opus*

In the insurance industry, customer support plays a crucial role in ensuring client satisfaction and retention. Insurance companies receive a high volume of support tickets daily, covering a wide range of topics such as billing, policy administration, claims assistance, and more. Manually categorizing these tickets can be time-consuming and inefficient, leading to longer response times and potentially impacting customer experience.

### Task
In this exercise, we will evaluate the accuracy of various prompts on the test.tsv dataset.

#### Labeled Data

The data can be found in the week-1 `data` folder.

We will use the following datasets:
- `./data/train.tsv`
- `./data/test.tsv`

In [4]:
training_examples = pd.read_csv('data/train.tsv', sep='\t')
test_examples = pd.read_csv('data/test.tsv', sep='\t')

# In order to not leak information about the test labels into our prompts, the list of possible categories will be defined 
# based on the training labels. We'll discuss train/test splits more during week 2.
categories = sorted(training_examples['label'].unique().tolist())
categories_str = '\n'.join(categories)

test_tickets = test_examples['text'].tolist()
test_labels = test_examples['label'].tolist()

In [5]:
idx = random.choice(range(len(training_examples["text"])))
print(training_examples["text"][idx], training_examples["label"][idx], sep = "\n")

I'm considering moving my insurance policies to your firm. Is it possible for you to provide a quote that offers the same or better coverage compared to what I currently have?
Quotes and Proposals


In [6]:
Counter(training_examples["label"]).most_common()

[('Billing Disputes', 9),
 ('General Inquiries', 9),
 ('Coverage Explanations', 8),
 ('Claims Disputes', 8),
 ('Billing Inquiries', 6),
 ('Policy Administration', 6),
 ('Claims Assistance', 6),
 ('Account Management', 6),
 ('Quotes and Proposals', 5),
 ('Policy Comparisons', 5)]

In [7]:
# Uses an LLM to predicted class labels for a list of support tickets
def classify_tickets(tickets, prompt_generator, model="accounts/fireworks/models/llama-v3p1-8b-instruct"):
    responses = list()

    for ticket in tickets:
        user_prompt = prompt_generator(ticket)
    
        response = client.chat.completions.create(
            model=model,
            messages=[
                { "role": "user", "content": user_prompt}
            ],
            # setting temperature to 0 for this use case, so that responses are as deterministic as possible
            temperature=0, 
            stop=["</category>"],
            max_tokens=2048,
        )
        response = response.choices[0].message.content.split("<category>")[1].strip()
        responses.append(response)

    return responses


# Calculates the percent of predictions we classified correctly
def evaluate_accuracy(predicted, actual):
    num_correct = sum([predicted[i] == actual[i] for i in range(len(actual))])
    return round(100 * num_correct / len(actual), 2)

### Classification with a simple prompt

We will first evaluate the accuracy of the LLM on a simple prompt that does not used any advanced prompt engineering techniques.

In [8]:
def create_simple_prompt(ticket):
    return f"""classify a customer support ticket into one of the following categories:
<categories>
{categories_str}
</categories>

Here is the customer support ticket:    
<ticket>{ticket}</ticket>

Respond using this format:
<category>The category label you chose goes here</category>"""    

In [9]:
simple_responses = classify_tickets(
    tickets=test_tickets, 
    prompt_generator=create_simple_prompt
)

In [11]:
with open("simple_responses_results.json", "w") as f:
    json.dump({"responses": simple_responses, "labels": test_labels}, f)

In [12]:
accuracy = evaluate_accuracy(simple_responses, test_labels)
print(f"Initial Prompt Accuracy: {accuracy}%")

Initial Prompt Accuracy: 54.41%


### Classification with an improved prompt

This prompt builds upon the simple prompt, and improves the accuracy of the classification by applying the following techniques:
- chain-of-thought: makes the LLM reflect on its reasoning before providing a response
- few-shot learning: provides examples within the context of the prompt

In [13]:
# Retrieve one example from each of the k most popular labels
def retrieve_few_shot_examples(df, k=5):
    # Count the frequency of each label
    label_counts = df['label'].value_counts()

    # Identify the k most common labels
    top_labels = label_counts.head(k).index

    # Retrieve a single row for each of these labels
    rows = df[df['label'].isin(top_labels)].groupby('label').first().reset_index()

    # Convert each row to the example string format required by the prompt
    example_strs = list()
    for idx, row in rows.iterrows():
        example_strs.append(f"<example><ticket>{row['text']}</ticket><label>{row['label']}</label></example>")

    return '\n'.join(example_strs)

few_shot_examples = retrieve_few_shot_examples(training_examples)

def create_improved_prompt(ticket):
    return f"""classify a customer support ticket into one of the following categories:
<categories>
{categories_str}
</categories>

Here is the customer support ticket:    
<ticket>{ticket}</ticket>

Use the following examples to help you classify the query:
<examples>
{few_shot_examples}
</examples>

First you will think step-by-step about the problem in the scratchpad tag.
You should consider all the information provided and create a concrete argument for your classification.

Respond using this format:
<response>
  <scratchpad>Your thoughts and analysis go here</scratchpad>
  <category>The category label you chose goes here</category>
</response>"""    

In [14]:
improved_responses = classify_tickets(
    tickets=test_tickets, 
    prompt_generator=create_improved_prompt
)

In [15]:
accuracy = evaluate_accuracy(improved_responses, test_labels)
print(f"Improved Prompt Accuracy: {accuracy}%")

Improved Prompt Accuracy: 70.59%


In [16]:
with open("improved_responses_results.json", "w") as f:
    json.dump({"responses": improved_responses, "labels": test_labels}, f)

# Send one example per label instead of only sending 1 of the top class

In [21]:
# Retrieve one example from each of the labels
def retrieve_few_shot_examples(df):
    # Retrieve a single row for each of these labels
    # Randomize the selection instead of picking the first one from each group
    rows = df.groupby('label').sample(1, random_state = 42).reset_index()

    # Convert each row to the example string format required by the prompt
    example_strs = list()
    for idx, row in rows.iterrows():
        example_strs.append(f"<example><ticket>{row['text']}</ticket><label>{row['label']}</label></example>")
    return '\n'.join(example_strs)

In [22]:
few_shot_examples = retrieve_few_shot_examples(training_examples)

def create_improved_prompt(ticket):
    return f"""classify a customer support ticket into one of the following categories:
<categories>
{categories_str}
</categories>

Here is the customer support ticket:    
<ticket>{ticket}</ticket>

Use the following examples to help you classify the query:
<examples>
{few_shot_examples}
</examples>

First you will think step-by-step about the problem in the scratchpad tag.
You should consider all the information provided and create a concrete argument for your classification.

Respond using this format:
<response>
  <scratchpad>Your thoughts and analysis go here</scratchpad>
  <category>The category label you chose goes here</category>
</response>"""    

In [23]:
second_test_responses = classify_tickets(
    tickets=test_tickets, 
    prompt_generator=create_improved_prompt
)

In [24]:
accuracy = evaluate_accuracy(second_test_responses, test_labels)
print(f"Improved Prompt Accuracy: {accuracy}%")

Improved Prompt Accuracy: 75.0%


In [25]:
df = pd.DataFrame({"ground_truth": test_labels, "simple_responses": simple_responses, 
                   "improved_responses": improved_responses, "one_example_per_class_responses": second_test_responses})

In [35]:
for i in range(1, 4):
    print(f"Performance of {df.columns[i]}\n")
    trues = df.iloc[:, 0]
    preds = df.iloc[:, i]
    print(classification_report(trues, preds, zero_division=1))

Performance of simple_responses

                       precision    recall  f1-score   support

   Account Management       1.00      0.33      0.50         6
     Billing Disputes       1.00      0.11      0.20         9
    Billing Inquiries       0.38      0.83      0.53         6
    Claims Assistance       0.50      0.50      0.50         6
      Claims Disputes       1.00      0.67      0.80         9
Coverage Explanations       0.45      1.00      0.62         9
    General Inquiries       1.00      0.00      0.00         7
Policy Administration       0.50      0.83      0.62         6
   Policy Comparisons       0.56      1.00      0.71         5
 Quotes and Proposals       1.00      0.20      0.33         5

             accuracy                           0.54        68
            macro avg       0.74      0.55      0.48        68
         weighted avg       0.75      0.54      0.48        68

Performance of improved_responses

                       precision    recall  f1-

In [39]:
# See the bottom 5 labels based on frequenct
training_examples.label.value_counts().tail(5)

label
Policy Administration    6
Claims Assistance        6
Account Management       6
Quotes and Proposals     5
Policy Comparisons       5
Name: count, dtype: int64

Seems like adding one example per class is giving better performance on bottom 5 classes without harming the top 5 classes performance too much...